<a href="https://colab.research.google.com/github/SSenitha/Data_Analytics/blob/main/masterNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analysis of Rain in Australia
This notebook contains all the steps of the analysis: From data cleaning and preprocessing to the conclusions.